In [2]:
import numpy as np
import pandas as pd
from dataclasses import dataclass
from math import log, sqrt, exp
from scipy.stats import norm

# -------------------------
# 1) Black-Scholes (Euro) Call
# -------------------------
def bs_call_price(S, K, T, r, q, sigma):
    if T <= 0:
        return max(S - K, 0.0)
    if sigma <= 0:
        return max(S*exp(-q*T) - K*exp(-r*T), 0.0)
    d1 = (log(S/K) + (r - q + 0.5*sigma*sigma)*T) / (sigma*sqrt(T))
    d2 = d1 - sigma*sqrt(T)
    return S*exp(-q*T)*norm.cdf(d1) - K*exp(-r*T)*norm.cdf(d2)

def bs_call_greeks(S, K, T, r, q, sigma):
    if T <= 0 or sigma <= 0:
        price = max(S-K,0.0)
        return dict(price=price, delta=float(S>K), gamma=0.0, theta=0.0, vega=0.0)
    d1 = (log(S/K) + (r - q + 0.5*sigma*sigma)*T) / (sigma*sqrt(T))
    d2 = d1 - sigma*sqrt(T)
    pdf = norm.pdf(d1)
    delta = exp(-q*T)*norm.cdf(d1)
    gamma = exp(-q*T)*pdf/(S*sigma*sqrt(T))
    vega  = S*exp(-q*T)*pdf*sqrt(T)
    # theta here is per year; convert to per day as theta/252 if you want
    theta = (-S*exp(-q*T)*pdf*sigma/(2*sqrt(T))
             - (r*K*exp(-r*T)*norm.cdf(d2))
             + (q*S*exp(-q*T)*norm.cdf(d1)))
    price = bs_call_price(S,K,T,r,q,sigma)
    return dict(price=price, delta=delta, gamma=gamma, theta=theta, vega=vega)

def implied_vol_call(price_mkt, S, K, T, r, q, lo=1e-6, hi=5.0, tol=1e-6, it=100):
    # simple bisection
    for _ in range(it):
        mid = 0.5*(lo+hi)
        pmid = bs_call_price(S,K,T,r,q,mid)
        if abs(pmid - price_mkt) < tol:
            return mid
        if pmid > price_mkt:
            hi = mid
        else:
            lo = mid
    return 0.5*(lo+hi)

# -------------------------
# 2) Monte Carlo price paths (GBM)
# -------------------------
def simulate_gbm_paths(S0, mu, vol, n_days, n_sims, dt=1/252, seed=0):
    rng = np.random.default_rng(seed)
    Z = rng.normal(size=(n_sims, n_days))
    # logS_{t+1} = logS_t + (mu - 0.5 vol^2)dt + vol sqrt(dt) Z
    drift = (mu - 0.5*vol*vol)*dt
    shock = vol*sqrt(dt)*Z
    logS = np.cumsum(drift + shock, axis=1)
    S = S0*np.exp(logS)
    return S  # shape (n_sims, n_days)

# -------------------------
# 3) Evaluate one option under scenarios
# -------------------------
@dataclass
class OptionSpec:
    K: float
    T_days: int     # time to expiry in trading days
    iv: float       # implied vol used for marking
    premium: float  # cost per share (option mid) in USD
    name: str = ""

def evaluate_long_call_mc(
    S0, opt: OptionSpec,
    mu, vol_realized,           # subjective drift, realized vol for price path
    r=0.03, q=0.00,
    n_sims=20000,
    hold_days=40,               # your 1-2m thesis horizon ~40d
    iv_scenario="flat",         # "flat" | "down" | "up"
    iv_shift=0.0,               # e.g. -0.05 for crush, +0.05 for spike
    seed=0
):
    n_days = min(hold_days, opt.T_days)  # don't go past expiry
    paths = simulate_gbm_paths(S0, mu, vol_realized, n_days=n_days, n_sims=n_sims, seed=seed)

    # mark option at exit (day n_days)
    S_exit = paths[:, -1]
    T_exit = max((opt.T_days - n_days)/252, 0.0)

    if iv_scenario == "flat":
        iv_exit = opt.iv
    elif iv_scenario == "down":
        iv_exit = max(opt.iv + iv_shift, 1e-6)
    elif iv_scenario == "up":
        iv_exit = max(opt.iv + iv_shift, 1e-6)
    else:
        raise ValueError("iv_scenario invalid")

    # option value at exit
    V_exit = np.array([bs_call_price(s, opt.K, T_exit, r, q, iv_exit) for s in S_exit])

    # P&L per share
    pnl_per_share = V_exit - opt.premium
    # per contract (100 shares)
    pnl_per_contract = 100*pnl_per_share

    out = {
        "name": opt.name,
        "K": opt.K,
        "T_days": opt.T_days,
        "premium_per_contract": 100*opt.premium,
        "iv_entry": opt.iv,
        "iv_exit": iv_exit,
        "hold_days": n_days,
        "P_profit": float((pnl_per_contract > 0).mean()),
        "p10": float(np.quantile(pnl_per_contract, 0.10)),
        "p50": float(np.quantile(pnl_per_contract, 0.50)),
        "p90": float(np.quantile(pnl_per_contract, 0.90)),
        "ES_05": float(pnl_per_contract[pnl_per_contract <= np.quantile(pnl_per_contract, 0.05)].mean()),
        "mean": float(pnl_per_contract.mean()),
    }
    return out

# -------------------------
# 4) Grid search (DTE x Delta proxy via K grid)
# -------------------------
def rank_candidates(candidates, key="p50"):
    return sorted(candidates, key=lambda x: x[key], reverse=True)


In [3]:
# --- Inputs from you ---
S0 = 84.5
K = 85.0
iv_entry = 0.675  # 67.5%

# If FUTU "premium 16.25%" means premium/underlying:
premium_per_share = 0.1625 * S0   # ~= 13.73 USD per share
# If you have the real option price in USD/share, overwrite premium_per_share.

# --- DTE (trading days) approximation ---
# simplest approximation (good enough for MC): trading_days ≈ 252 * calendar_year_fraction
today = pd.Timestamp("2026-01-14")
expiry = pd.Timestamp("2026-05-15")
T_days = int(round(252 * (expiry - today).days / 365.25))

from dataclasses import dataclass
@dataclass
class OptionSpec:
    K: float
    T_days: int
    iv: float
    premium: float
    name: str = ""

opt = OptionSpec(K=K, T_days=T_days, iv=iv_entry, premium=premium_per_share, name="SLV May15 85C")
print(opt)

OptionSpec(K=85.0, T_days=83, iv=0.675, premium=13.731250000000001, name='SLV May15 85C')


In [4]:
r = 0.03
q = 0.00
T = opt.T_days / 252

g = bs_call_greeks(S0, opt.K, T, r, q, opt.iv)
print("BS greeks:", g)

theta_per_day_per_share = g["theta"] / 252
theta_per_day_per_contract = 100 * theta_per_day_per_share
print("Theta per day per contract (approx):", theta_per_day_per_contract)


BS greeks: {'price': np.float64(13.12055005172548), 'delta': np.float64(0.580811367058439), 'gamma': np.float64(0.011936491771567644), 'theta': np.float64(-20.495093840860523), 'vega': np.float64(18.948352060367885)}
Theta per day per contract (approx): -8.132973746373223


In [11]:
def simulate_gbm_paths(S0, mu, vol, n_days, n_sims, dt=1/252, seed=0):
    rng = np.random.default_rng(seed)
    Z = rng.normal(size=(n_sims, n_days))
    drift = (mu - 0.5*vol*vol)*dt
    shock = vol*np.sqrt(dt)*Z
    logS = np.cumsum(drift + shock, axis=1)
    return S0*np.exp(logS)

def evaluate_long_call_mc(
    S0, opt: OptionSpec,
    mu, vol_realized,
    r=0.03, q=0.00,
    n_sims=20000,
    hold_days=40,
    iv_exit=None,
    seed=0
):
    n_days = min(hold_days, opt.T_days)
    paths = simulate_gbm_paths(S0, mu, vol_realized, n_days=n_days, n_sims=n_sims, seed=seed)

    S_exit = paths[:, -1]
    T_exit = max((opt.T_days - n_days)/252, 0.0)

    # choose IV for marking at exit
    if iv_exit is None:
        iv_exit = opt.iv

    V_exit = np.array([bs_call_price(s, opt.K, T_exit, r, q, iv_exit) for s in S_exit])

    pnl_per_contract = 100*(V_exit - opt.premium)

    out = {
        "mu": mu, "vol_realized": vol_realized, "iv_exit": iv_exit,
        "P_profit": float((pnl_per_contract > 0).mean()),
        "p10": float(np.quantile(pnl_per_contract, 0.10)),
        "p50": float(np.quantile(pnl_per_contract, 0.50)),
        "p90": float(np.quantile(pnl_per_contract, 0.90)),
        "mean": float(pnl_per_contract.mean()),
    }
    return out

def mu_from_expected_return(R, days=21, trading_days=252):
    T = days / trading_days
    return np.log(1 + R) / T

mu_grid = [0.4, 0.6, 0.8, 1, 1.2]
vol_grid = [0.6, 0.65, 0.7, 0.75, 0.8]
iv_cases = {
    "flat": opt.iv,
    "down_10vol": max(opt.iv - 0.10, 1e-6),
    "up_10vol": opt.iv + 0.10,
}

results = []
for mu in mu_grid:
    for vol in vol_grid:
        for name, ivx in iv_cases.items():
            res = evaluate_long_call_mc(S0, opt, mu=mu, vol_realized=vol, hold_days=40, iv_exit=ivx, n_sims=30000, seed=1)
            res["iv_case"] = name
            results.append(res)

df = pd.DataFrame(results).sort_values(["mu","vol_realized","iv_case"])

     mu  vol_realized  iv_exit  P_profit          p10         p50  \
1   0.4          0.60    0.575  0.390733 -1267.061397 -398.887589   
0   0.4          0.60    0.675  0.426500 -1199.067966 -259.541641   
2   0.4          0.60    0.775  0.464600 -1120.531371 -120.260130   
4   0.4          0.65    0.575  0.391567 -1293.249271 -424.479393   
3   0.4          0.65    0.675  0.424967 -1234.712179 -285.136808   
..  ...           ...      ...       ...          ...         ...   
69  1.2          0.75    0.675  0.584567 -1151.932677  440.225707   
71  1.2          0.75    0.775  0.614533 -1063.943406  571.726692   
73  1.2          0.80    0.575  0.545900 -1265.564603  268.480106   
72  1.2          0.80    0.675  0.571100 -1197.079711  396.255081   
74  1.2          0.80    0.775  0.600267 -1118.105654  528.639792   

            p90         mean     iv_case  
1   2173.134953    89.444883  down_10vol  
0   2240.648288   191.321347        flat  
2   2325.214292   299.883605    up_10vol  

In [13]:
# 2) Focus on the mu band that matches your belief (~10% / month => mu ~ 1.14 annualized)
bull = df[df["mu"].between(1.0, 1.2)]

# 3) “Robust within bullish band”: what’s the range across vol/IV scenarios?
robust = bull.agg(
    P_profit_min=("P_profit","min"),
    P_profit_med=("P_profit","median"),
    P_profit_max=("P_profit","max"),
    p50_min=("p50","min"),
    p50_med=("p50","median"),
    p50_max=("p50","max"),
    p10_min=("p10","min"),
    p90_med=("p90","median"),
    mean_med=("mean","median"),
)
print("\nRobust stats for mu in [1.0, 1.2]:\n", robust)

# 4) Pick a base scenario (example): vol_realized=0.70, iv_exit=0.675 (flat IV)
base = df[(df.mu==1.0) & (df.vol_realized==0.70) & (df.iv_exit==0.675)]
print("\nBase (mu=1.0, vol=0.70, IV flat):\n", base[["P_profit","p10","p50","p90","mean"]])

# 5) Conservative scenario (same mu, but IV crush): iv_exit=0.575
cons = df[(df.mu==1.0) & (df.vol_realized==0.70) & (df.iv_exit==0.575)]
print("\nConservative (mu=1.0, vol=0.70, IV -10vol):\n", cons[["P_profit","p10","p50","p90","mean"]])

# 6) Optimistic scenario (same mu, IV expands): iv_exit=0.775
opt = df[(df.mu==1.0) & (df.vol_realized==0.70) & (df.iv_exit==0.775)]
print("\nOptimistic (mu=1.0, vol=0.70, IV +10vol):\n", opt[["P_profit","p10","p50","p90","mean"]])


Robust stats for mu in [1.0, 1.2]:
               P_profit         p50          p10          p90         mean
P_profit_min  0.507633         NaN          NaN          NaN          NaN
P_profit_med  0.576417         NaN          NaN          NaN          NaN
P_profit_max  0.667633         NaN          NaN          NaN          NaN
p50_min            NaN   43.541647          NaN          NaN          NaN
p50_med            NaN  373.734648          NaN          NaN          NaN
p50_max            NaN  687.043427          NaN          NaN          NaN
p10_min            NaN         NaN -1292.817827          NaN          NaN
p90_med            NaN         NaN          NaN  4109.354034          NaN
mean_med           NaN         NaN          NaN          NaN  1020.324947

Base (mu=1.0, vol=0.70, IV flat):
     P_profit          p10         p50          p90        mean
51  0.554267 -1152.227664  255.830785  3890.335067  898.040339

Conservative (mu=1.0, vol=0.70, IV -10vol):
 Empty DataFrame